# Hill Climbing Method
Implement and train hill climbing with adaptive noise scaling with OpenAI Gym's Cartpole environment

## Pseudo code

```
Initialize the weights 𝜃 in the policy arbitarily.

Collect an episode with 𝜃, and record the return G.

𝜃_best <-- 𝜃, G_best <-- G

Repeat until environment solved:

    Add a little bit of random noise to 𝜃_best, to get a new set of weight 𝜃_new.

    Collect an episode with 𝜃_new, and record the return G_new.
    
    if G_new < G_best, then:
        𝜃_best <-- 𝜃_new, G_best <-- G_new
```


### Import the Necessary Packages

In [1]:
import gym
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

from pyvirtualdisplay import Display

display = Display(visible=0, size=(1400, 900))
display.start()

is_ipython = 'inline' in plt.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

### Specify the Environment, and Explore the State and Action Spaces

Create an environment and set random seed

In [2]:
# Create an environment and set random seed
env = gym.make('CartPole-v1')
# env.seed(0);
np.random.seed(0)

# Explore state (observation) space
print("State space: ", env.observation_space)
print("State shape: ", env.observation_space.shape)
print("- low: ", env.observation_space.low)
print("- high: ", env.observation_space.high)
print("")
# Generate some samples from the state space 
print("State space samples: \n", np.array([env.observation_space.sample() for i in range(10)]))

print("\n\n")

# Explore the action space
print("Action space:", env.action_space)
print("Action shape:", env.action_space.shape)
# Generate some samples from the action space
print("Action space samples:")
print(np.array([env.action_space.sample() for i in range(10)]))

State space:  Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
State shape:  (4,)
- low:  [-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38]
- high:  [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]

State space samples: 
 [[-2.3792100e+00 -2.1373928e+38 -1.6146666e-01  2.4863155e+38]
 [ 1.7826372e+00 -8.0585842e+37 -4.1131487e-01  2.7576343e+38]
 [-1.2242954e+00  2.0758486e+38  3.7988812e-01  1.0728510e+38]
 [ 1.0862420e+00 -1.4877127e+38 -3.4904916e-02  5.3381938e+37]
 [-4.7873945e+00 -1.3616253e+38  5.9314738e-03 -9.8459652e+36]
 [-6.5114066e-02  1.2362231e+38 -3.4024039e-01  2.1570726e+38]
 [-2.8046293e+00 -2.9797363e+38  4.1451511e-01  2.9488303e+38]
 [ 4.2535138e+00 -2.3991322e+38  3.7499353e-01  1.2329292e+38]
 [-3.7097363e+00  2.8606872e+38  1.7501388e-01 -1.1100780e+38]
 [-3.7729321e+00  7.6547457e+37  2.3038036e-01  9.5956567e+37]]



Action space: Discrete(2)
Actio

### Define the Policy

In [3]:
class Policy():
    def __init__(self, s_size=4, a_size=2):
        self.w = 1e-4 * np.random.rand(s_size, a_size)     # weights for simple linear policy: state_sapce x action_space
        
    def forward(self, state):
        """
        Target labels used to train a model with a sigmoid activation function in 
        the output layer will have the values 0 or 1.

        Apply Softmax Output Activation Function
        """
        x = np.dot(state, self.w)
        return np.exp(x)/sum(np.exp(x))
    
    def act(self, state):
        probs = self.forward(state)
        
        #Choose case for stochastic policy or deterministic policy
        # action = np.random.choice(2, p=probs) # Stochastic policy
        action = np.argmax(probs) # Deterministic policy
        
        return action

### Train the Agent

In [4]:
policy = Policy()

In [5]:
def hill_climbing(n_episodes=1000, max_t=1000, gamma=1.0, print_every=100, noise_scale=1e-2):
    """Implementation of hill climbing with adaptive noise scaling.
       
       Psuedo code
       -----------
        Initialize the weights 𝜃 in the policy arbitarily.

        Collect an episode with 𝜃, and record the return G.

        𝜃_best <-- 𝜃, G_best <-- G

        Repeat until environment solved:

            Add a little bit of random noise to 𝜃_best, to get a new set of weight 𝜃_new.

            Collect an episode with 𝜃_new, and record the return G_new.

            if G_new < G_best, then:
                𝜃_best <-- 𝜃_new, G_best <-- G_new

    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        gamma (float): discount rate
        print_every (int): how often to print average score (over last 100 episodes)
        noise_scale (float): standard deviation of additive noise
    """
    
    # initialize
    scores_deque = deque(maxlen=100)
    scores = []
    best_R = -np.Inf
    best_w = policy.w
    
    
    for i_episode in range(1, n_episodes+1):
        rewards = []
        state = env.reset()
        for t in range(max_t):
            action = policy.act(state)
            state, reward, done, _ = env.step(action)
            rewards.append(reward)
            if done:
                break 
        scores_deque.append(sum(rewards))
        scores.append(sum(rewards))

        discounts = [gamma**i for i in range(len(rewards)+1)]
        R = sum([a*b for a,b in zip(discounts, rewards)])

        if R >= best_R: # found better weights
            best_R = R
            best_w = policy.w
            noise_scale = max(1e-3, noise_scale / 2)
            policy.w += noise_scale * np.random.rand(*policy.w.shape) 
        else: # did not find better weights
            noise_scale = min(2, noise_scale * 2)
            policy.w = best_w + noise_scale * np.random.rand(*policy.w.shape)

        # Monitor case
        if i_episode % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
        if np.mean(scores_deque)>=195.0:
            print('Environment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_deque)))
            policy.w = best_w
            break
        
    return scores
            
scores = hill_climbing()
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


ValueError: shapes (2,) and (4,2) not aligned: 2 (dim 0) != 4 (dim 0)

### Simulate Trained Agent

In [ ]:
env = gym.make('CartPole-v0')
state = env.reset()
img = plt.imshow(env.render(mode='rgb_array'))
for t in range(200):
    action = policy.act(state)
    img.set_data(env.render(mode='rgb_array')) 
    plt.axis('off')
    display.display(plt.gcf())
    display.clear_output(wait=True)
    state, reward, done, _ = env.step(action)
    if done:
        break 

env.close()

/usr/local/lib/python3.8/dist-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(


TypeError: render() got an unexpected keyword argument 'mode'